In [1]:
# 类似文本翻译
# 就是输入一个词,然后输出另一个词
不知道为什么效果很差，建议看另一个
来自两天后的更新捏
=============================================
翻转了，是反向传播，更新参数，梯度清零这三件套没写明白捏,这下贝极星了


In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
#S代表开始字符  E代表结束字符 ?代表无法识别的字符
letter = [c for c in 'SE?abcdefghijklmnopqrstuvwxyz']
mydata = [['man', 'women'], ['black', 'white'], ['king', 'queen'],
          ['girl', 'boy'], ['up', 'down'], ['high', 'low'],
          ['cat', 'dog'], ['hello', 'world'], ['good', 'bye']]
letter2index = {v: k for k, v in enumerate(letter)}
index2letter = {k: v for k, v in enumerate(letter)}
print(letter2index)
print(index2letter)

{'S': 0, 'E': 1, '?': 2, 'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
{0: 'S', 1: 'E', 2: '?', 3: 'a', 4: 'b', 5: 'c', 6: 'd', 7: 'e', 8: 'f', 9: 'g', 10: 'h', 11: 'i', 12: 'j', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'q', 20: 'r', 21: 's', 22: 't', 23: 'u', 24: 'v', 25: 'w', 26: 'x', 27: 'y', 28: 'z'}


In [4]:
# 每个单词都变为5个字母,不够的在后面补?
max_len = max([max(len(i), len(j)) for i, j in mydata])
print('每个单词变的维度:', max_len)  # 每个单词变的维度: 5
# print('每个单词变的维度:', n_step)  # 每个单词变的维度: 5

每个单词变的维度: 5


In [5]:
n_class = 29  # 26个字母加3个符号(SE?)
batch_size = 1

<img src="encoderdecoder.png"> <br>
- 下面是对数据进行处理，主要做的是
- 首先对单词长度不够的，用 '?' 进行填充
- 然后将 Deocder 的输入数据末尾添加终止标志 'E'
- Decoder 的输入数据开头添加开始标志 'S'
- Decoder 的输出数据末尾添加结束标志 'E'

In [6]:
def make_data(myData):
    encode_input_all = []
    decode_input_all = []
    decode_output_all = []
    # 1.首先对单词长度不够的，用 ‘?’ 进行填充
    for seq in myData:
        for i in range(2):
            seq[i] = seq[i] + '?' * (max_len - len(seq[i]))
        # print(seq)# ['man??', 'women'] ['black', 'white']...['up???', 'down?']
        encode_input = [letter2index[n] for n in (seq[0] + 'E')]
        decode_input = [letter2index[n] for n in ('S' + seq[1])]
        decode_output = [letter2index[n] for n in (seq[1]) + 'E']

        # 用np.eye(n_class) n_class是字典长度 转one-hot
        encode_input_all.append(np.eye(n_class)[encode_input])
        decode_input_all.append(np.eye(n_class)[decode_input])
        decode_output_all.append(decode_output)
    return torch.Tensor(encode_input_all), torch.Tensor(decode_input_all), torch.LongTensor(decode_output_all)



In [7]:
encode_input_all, decode_input_all, decode_output_all = make_data(mydata)

D:\ProgramData\Anaconda3\envs\pycharmshiyong\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)


In [8]:
print(encode_input_all, type(encode_input_all), len(encode_input_all))
print("-----------")
print(encode_input_all[1], encode_input_all[1].shape)
#torch.Size([6, 29]) 6代表5+1,,29表示每个字符29维

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0

In [9]:
class myDataset(Data.Dataset):
    def __init__(self, encode_input_all, decode_input_all, decode_output_all):
        self.encode_input_all = encode_input_all
        self.decode_input_all = decode_input_all
        self.decode_output_all = decode_output_all

    def __getitem__(self, item):
        return self.encode_input_all[item], self.decode_input_all[item], self.decode_output_all[item]

    def __len__(self):
        return len(self.encode_input_all)


dataset = myDataset(encode_input_all, decode_input_all, decode_output_all)
# j = 0
# for i in dataset:
#     print(i)
#     j = j + 1
#     print(j)

In [10]:
dataloader = Data.DataLoader(dataset=dataset, batch_size=1, shuffle=False, drop_last=True)

In [11]:
for x, y, z in dataloader:
    print("x", x, x.shape)  # 0 1 2
    print("y", y, y.shape)
    print("z", z, z.shape)
    print('----------------')
    break

x tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]) torch.Size([1, 6, 29])
y tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0.

In [16]:
## model
class Seq2seq(nn.Module):
    def __init__(self):
        super(Seq2seq, self).__init__()
        ##n_hidden = 128
        self.encoder = nn.RNN(input_size=n_class, hidden_size=128, dropout=0.5)
        self.decoder = nn.RNN(input_size=n_class, hidden_size=128, dropout=0.5)
        self.fc = nn.Linear(in_features=128, out_features=n_class)

    def forward(self, encode_input, decode_input):
        batch_size = 1
        encode_input = encode_input.transpose(0, 1)
        decode_input = decode_input.transpose(0, 1)
        h_0 = torch.zeros(1, batch_size, 128)
        out1, h_n1 = self.encoder(encode_input, h_0)
        out2, h_n2 = self.decoder(decode_input, h_n1)
        # out2 = out2[-1]  ## out2本身就只有一个输出
        model = self.fc(out2)
        # print(model.shape)
        return model

In [17]:
model = Seq2seq()
print(model)

Seq2seq(
  (encoder): RNN(29, 128, dropout=0.5)
  (decoder): RNN(29, 128, dropout=0.5)
  (fc): Linear(in_features=128, out_features=29, bias=True)
)


In [18]:
## 损失函数&&优化器
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [19]:
for epoch in range(500):
    for x, y, z in dataloader:
        pred = model(x, y)
        pred = pred.transpose(0, 1)
        # print(pred.shape)
        loss = 0
        for i in range(len(z)):
            loss += loss_fn(pred[i], z[i])
        print(epoch, loss.item())

        ###
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()

0 3.353398084640503
0 3.4036448001861572
0 3.379035711288452
0 3.4176864624023438
0 3.4133176803588867
0 3.4254491329193115
0 3.3807003498077393
0 3.4087793827056885
0 3.3998165130615234
1 3.353398084640503
1 3.4036448001861572
1 3.379035711288452
1 3.4176864624023438
1 3.4133176803588867
1 3.4254491329193115
1 3.3807003498077393
1 3.4087793827056885
1 3.3998165130615234
2 3.353398084640503
2 3.4036448001861572
2 3.379035711288452
2 3.4176864624023438
2 3.4133176803588867
2 3.4254491329193115
2 3.3807003498077393
2 3.4087793827056885
2 3.3998165130615234
3 3.353398084640503
3 3.4036448001861572
3 3.379035711288452
3 3.4176864624023438
3 3.4133176803588867
3 3.4254491329193115
3 3.3807003498077393
3 3.4087793827056885
3 3.3998165130615234
4 3.353398084640503
4 3.4036448001861572
4 3.379035711288452
4 3.4176864624023438
4 3.4133176803588867
4 3.4254491329193115
4 3.3807003498077393
4 3.4087793827056885
4 3.3998165130615234
5 3.353398084640503
5 3.4036448001861572
5 3.379035711288452
5 3.

In [20]:
def test(word):
    encode_input, decode_input, _ = make_data([[word, '?' * len(word)]])
    output = model(encode_input, decode_input)
    # output.data.max(2, keepdim=True)[1]中的2表示，找第3维的最大值；
    # [1]表示，output.data.max(2, keepdim=True)会返回一个数组，
    # 第一个是output数组中第3维度的最大值是多少，
    # 第二个是最大值的位置在哪里。
    predict = output.data.max(2, keepdim=True)[1]
    decode = [letter[i] for i in predict]
    trans = ''.join((decode))
    return trans

In [21]:

print('man->',test('man'))
print('hello->',test('hello'))

man-> xggggg
hello-> xggggg


['man??', 'women']
['black', 'white']
['king?', 'queen']


<class 'list'>
